In [16]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

df=pd.read_csv('clean_a-warhol_data.csv',sep=None)
df=df.drop(columns="Unnamed: 0")
for i in df.index:
    try:
        df.loc[i,'hammer_price']=int(df.loc[i,'hammer_price'])
    except:
        df=df.drop(i)
        continue
    try:
        df.loc[i,'year']=int(df.loc[i,'year'])
    except:
        df=df.drop(i)
        continue
    try:
        df.loc[i,'size_dim1']=float(df.loc[i,'size_dim1'])
    except:
        df=df.drop(i)
        continue

/Users/home/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x1092bfa20>>
Traceback (most recent call last):
  File "/Users/home/anaconda3/envs/py36/lib/python3.6/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x109328198>>
Traceback (most recent call last):
  File "/Users/home/anaconda3/envs/py36/lib/python3.6/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [17]:
# XGBoost preprocessing
# REMOVING COLUMNS THAT XGBOOST CANNOT HANDLE
df=df.drop(columns='title_piece')
df=df.drop(columns='auction_date')

# Splitting data
X = df.drop(columns="hammer_price")
y = df['hammer_price']

#ONE HOT ENCODING CATEGORICAL DATA
X=pd.concat([X,pd.get_dummies(X['year'],prefix='year')],axis=1).drop(['year'],axis=1)
X=pd.concat([X,pd.get_dummies(X['auction_name'],prefix='auction_name')],axis=1).drop(['auction_name'],axis=1)
X=pd.concat([X,pd.get_dummies(X['auction_location'],prefix='auction_location')],axis=1).drop(['auction_location'],axis=1)
X=pd.concat([X,pd.get_dummies(X['art_type'],prefix='art_type')],axis=1).drop(['art_type'],axis=1)
X=pd.concat([X,pd.get_dummies(X['material1'],prefix='material1')],axis=1).drop(['material1'],axis=1)
X=pd.concat([X,pd.get_dummies(X['material2'],prefix='material2')],axis=1).drop(['material2'],axis=1)
X=pd.concat([X,pd.get_dummies(X['material3'],prefix='material3')],axis=1).drop(['material3'],axis=1)
X=pd.concat([X,pd.get_dummies(X['size_dim1'],prefix='size_dim1')],axis=1).drop(['size_dim1'],axis=1)
#X=pd.concat([X,pd.get_dummies(X['material4'],prefix='material4')],axis=1).drop(['material4'],axis=1)
#df=pd.concat([df,pd.get_dummies(df['material5'],prefix='material5')],axis=1).drop(['material5'],axis=1)

In [20]:
#XGBOOST Training Model
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)# Fitting XGBoost to the Training set

regressor = XGBRegressor()
regressor.fit(X_train, y_train)

# Predicting the Test set results
y_pred = regressor.predict(X_test)
print(mean_squared_error(y_test, y_pred))

200629774368.1239


In [ ]:
# Hyperparameter optimization: Gridsearch
print("Parameter optimization")
xgb_model = xgb.XGBRegressor()
clf = GridSearchCV(xgb_model,
                   {'max_depth': [2,3,4],
                    'n_estimators': [80,100,150]}, verbose=1)
clf.fit(X,y)
print(clf.best_score_)
print(clf.best_params_)

Parameter optimization
Fitting 3 folds for each of 9 candidates, totalling 27 fits
